# Importando os dados de bilheteria na ANCINE

A [ANCINE](https://www.gov.br/ancine/pt-br/oca/dados-abertos) (Agência Nacional do Cinema) disponibiliza em seu portal de dados públicos os valores diários das bilheterias de todos os cinemas do Brasil.

## Preparação dos arquivos

In [52]:
# Bibliotecas necessárias

import pandas as pd
import zipfile 
import requests, io, os
import glob

In [42]:
# Link com os dados que quero analisar referente aos filmes exibidos no Brasil

link_zip = 'https://dados.ancine.gov.br/dados-abertos/BilheteriaObrasDia.zip'

In [53]:
# Definindo o caminho onde os arquivos serão salvos

pasta = '/Users/bentofh/Documents/GitHub/ancine/Bilheteria/'

In [43]:
# Baixando o arquivo .zip e extraindo os arquivos .csv

getfolder = requests.get(link_zip)
unzipfolder = zipfile.ZipFile(io.BytesIO(getfolder.content))
unzipfolder.extractall(pasta)
os.chdir(pasta)
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

In [44]:
# Transformando os arquivos em parquet

anos = range(2014, 2024)

for ano in anos:
    arquivo_csv = os.path.join(pasta, f'BilheteriaObrasDia_{ano}.csv')
    if os.path.isfile(arquivo_csv):
        try:
            df = pd.read_csv(arquivo_csv, encoding='utf8', delimiter=';')
        except UnicodeDecodeError:
            df = pd.read_csv(arquivo_csv, encoding='ISO-8859-1', delimiter=';')
    arquivo_parquet = os.path.join(pasta, f'BilheteriaObrasDia_{ano}.parquet')
    df.to_parquet(arquivo_parquet)

In [45]:
# Apagar os arquivos .csv
    
arquivos_csv = glob.glob(os.path.join(pasta, '*.csv'))

for arquivo in arquivos_csv:
    os.remove(arquivo)

In [82]:
# Lista para armazenar os dataframes
dataframes = []

ano_min_analise = 2014
ano_max_analise = 2024

# Itera sobre os arquivos na pasta
for ano in range(ano_min_analise, ano_max_analise):
    arquivo = os.path.join(pasta, f'BilheteriaObrasDia_{ano}.parquet')
    df = pd.read_parquet(arquivo)
    dataframes.append(df)

# Combina todos os dataframes em um único
df_total = pd.concat(dataframes, ignore_index=True)

In [84]:
# Gerando uma amostra 
df_amostra = df_total.sample(n=10000)

# Salva a amostra em parquet
df_amostra.to_parquet(os.path.join(pasta,r'BrilheteriaAmostra.parquet'))

# Salva a amostra em csv
df_amostra.to_csv(os.path.join(pasta,r'BrilheteriaAmostra.csv'), index=False)